## This part aim to pick some demo images.
* The demo images set has 1699 images.

In [1]:
import keras
from keras.applications import DenseNet121 #DenseNet121, DenseNet169, DenseNet201, ResNet50
from keras.models import Model
from keras.layers import Dense
from keras.regularizers import l2
import numpy as np
import matplotlib.pyplot as plt
from label_dict import label_dict

/opt/conda/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#Build the new model based on DenseNet121
def getmodel(num_classes=256):
    '''
    Build the new model based on DenseNet121, DenseNet169, DenseNet201
    
    Argument:
    num_classes -- (int) number of classes
    
    Return:
    model -- the new compiled model
    '''
    
    orig_model = DenseNet121(include_top=False, weights='imagenet', pooling='avg', classes=num_classes)
    densenet_out = orig_model.layers[-1].output
    out = Dense(256, activation='softmax', kernel_initializer='glorot_uniform')(densenet_out)
    model = Model(input=orig_model.input, output=out)
    model.compile(loss= "sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    return model

In [3]:
model = getmodel(256)

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  app.launch_new_instance()


In [4]:
#Import test dataset
X_test = np.load('test_resize_221.npy')/225.
y_test = np.load('test.npy')

In [5]:
model.load_weights('DenseNet121_transfer.hdf5')

In [6]:
#evaluate
eva = model.evaluate(X_test, y_test[:, 0], batch_size=32, verbose=1, sample_weight=None)
print()
print("Loss = " + str(eva[0]))
print("Test Accuracy = " + str(eva[1]))

5956/5956 [==============================] - 31s 5ms/step
()
Loss = 2.684517687643191
Test Accuracy = 0.6121558092679651


In [7]:
index = np.max(model.predict(X_test), axis=1, keepdims=True) >= 0.9999

In [8]:
X_test = np.load('test_resize_221.npy')

In [9]:
demo_img = X_test[index.T.tolist()[0]]
demo_label = y_test[:, 0][index.T.tolist()[0]]

In [10]:
print('The number of demo images is: '+str(demo_img.shape[0]))

The number of demo images is: 1699


In [11]:
np.save('demo_img.npy', demo_img)
np.save('demo_label.npy', demo_label)